<font size = '6' >**Loading packages**</font>

In [1]:
import pandas as pd
import numpy as np
import scipy
import pickle
import os
import gc
import ast
from glob import glob
import zlib
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.notebook import tqdm
from PIL import Image
import cv2
import torch
import tensorflow as tf
import seaborn as sns
import random
import tensorflow.keras.backend as K


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

!pip install /kaggle/input/kerasapplications -q
!pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps
!pip install -q "/kaggle/input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl"
!pip install -q "/kaggle/input/hpapytorchzoozip/pytorch_zoo-master"
!pip install -q "/kaggle/input/hpacellsegmentatormaster/HPA-Cell-Segmentation-master"
import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell
from pycocotools import _mask as coco_mask
import efficientnet.keras as efn

In [2]:
# PRESETS
INT_2_STR = {0: 'Nucleoplasm', 
             1: 'Nuclear Membrane', 
             2: 'Nucleoli', 
             3: 'Nucleoli Fibrillar Center', 
             4: 'Nuclear Speckles', 
             5: 'Nuclear Bodies', 
             6: 'Endoplasmic Reticulum', 
             7: 'Golgi Apparatus', 
             8: 'Intermediate Filaments', 
             9: 'Actin Filaments', 
             10: 'Microtubules', 
             11: 'Mitotic Spindle', 
             12: 'Centrosome', 
             13: 'Plasma Membrane', 
             14: 'Mitochondria', 
             15: 'Aggresome', 
             16: 'Cytosol', 
             17: 'Vesicles', 
             18: 'Negative'}

<font size = '6' >**Define the paths**</font>

In [3]:
# Define paths to nucleus and cell models for the cellsegmentator class
NUC_MODEL = '/kaggle/input/hpacellsegmentatormodelweights/dpn_unet_nuclei_v1.pth'
CELL_MODEL = '/kaggle/input/hpacellsegmentatormodelweights/dpn_unet_cell_3ch_v1.pth'

#main_model = "../input/xception-model/ckpt-0008-0.0595.ckpt"
auxiliary_model = "../input/efnb7-model/efbn7_model_20_green.h5"
main_model = "../input/xception-model/final/0004-0.0652.ckpt"

# Define the path to the competition data directory
DATA_DIR = "/kaggle/input/hpa-single-cell-image-classification"

# Define the paths to the training and testing tfrecord and 
# image folders respectively for the competition data
TEST_IMG_DIR = os.path.join(DATA_DIR, "test")

# Capture all the relevant full image paths for the competition dataset
TEST_IMG_PATHS = sorted([os.path.join(TEST_IMG_DIR, f_name) for f_name in os.listdir(TEST_IMG_DIR)])
print(f"... The number of testing images is {len(TEST_IMG_PATHS)}" \
      f"\n\t--> i.e. {len(TEST_IMG_PATHS)//4} 4-channel images ...")

# Define paths to the relevant csv files
PUB_SS_CSV = "/kaggle/input/hpa-sample-submission-with-extra-metadata/updated_sample_submission.csv"
SWAP_SS_CSV = os.path.join(DATA_DIR, "sample_submission.csv")

# Create the relevant dataframe objects
ss_df = pd.read_csv(SWAP_SS_CSV)

# Test Time Augmentation Information
DO_TTA = True
TTA_REPEATS = 8

# helps us control whether this is the full submission or just the initial pass
IS_DEMO = len(ss_df)==559

if IS_DEMO:
    ss_df_1 = ss_df.drop_duplicates("ImageWidth", keep="first")
    ss_df_2 = ss_df.drop_duplicates("ImageWidth", keep="last")
    ss_df = pd.concat([ss_df_1, ss_df_2])
    del ss_df_1; del ss_df_2; gc.collect();
    print("\n\nSAMPLE SUBMISSION DATAFRAME\n\n")
    display(ss_df)
else:
    print("\n\nSAMPLE SUBMISSION DATAFRAME\n\n")
    display(ss_df)
    
# If demo-submission/display we only do a subset of the data
ONLY_PUBLIC = True
if ONLY_PUBLIC:
    pub_ss_df = pd.read_csv(PUB_SS_CSV)
    
    if IS_DEMO:
        pub_ss_df_1 = pub_ss_df.drop_duplicates("ImageWidth", keep="first")
        pub_ss_df_2 = pub_ss_df.drop_duplicates("ImageWidth", keep="last")
        pub_ss_df = pd.concat([pub_ss_df_1, pub_ss_df_2])
        
    pub_ss_df.mask_rles = pub_ss_df.mask_rles.apply(lambda x: ast.literal_eval(x))
    pub_ss_df.mask_bboxes = pub_ss_df.mask_bboxes.apply(lambda x: ast.literal_eval(x))
    pub_ss_df.mask_sub_rles = pub_ss_df.mask_sub_rles.apply(lambda x: ast.literal_eval(x))
    
    print("\n\nTEST DATAFRAME W/ MASKS\n\n")
    display(pub_ss_df)    

... The number of testing images is 2236
	--> i.e. 559 4-channel images ...


SAMPLE SUBMISSION DATAFRAME




,ID,ImageWidth,ImageHeight,PredictionString
0,0040581b-f1f2-4fbe-b043-b6bfea5404bb,2048,2048,0 1 eNoLCAgIMAEABJkBdQ==
4,0173029a-161d-40ef-af28-2342915b22fb,3072,3072,0 1 eNoLCAgIsAQABJ4Beg==
6,020a29cf-2c24-478b-8603-c22a90dc3e31,1728,1728,0 1 eNoLCAjJNgIABNkBkg==
554,fea47298-266a-4cf4-93bd-55d1bcc2fc7d,1728,1728,0 1 eNoLCAjJNgIABNkBkg==
557,ff069fa2-d948-408e-91b3-034cfea428d1,3072,3072,0 1 eNoLCAgIsAQABJ4Beg==
558,ff23eea9-4bbe-42af-a8da-9ae16321fc6d,2048,2048,0 1 eNoLCAgIMAEABJkBdQ==




TEST DATAFRAME W/ MASKS




,ID,ImageWidth,ImageHeight,PredictionString,mask_rles,mask_bboxes,mask_sub_rles
0,0040581b-f1f2-4fbe-b043-b6bfea5404bb,2048,2048,0 1 eNoLCAgIMAEABJkBdQ==,[729089 8 729109 12 731137 8 731157 12 733185 ...,"[(356, 0, 1076, 244), (764, 0, 1456, 756), (15...",[eNqtVL0OgyAQfiVIz4TBoYODiVdKE2odGDqYRi1p338rr...
4,0173029a-161d-40ef-af28-2342915b22fb,3072,3072,0 1 eNoLCAgIsAQABJ4Beg==,[1 558 3073 558 6145 558 9217 558 12289 558 15...,"[(0, 0, 672, 624), (258, 0, 882, 318), (882, 0...",[eNq9VMEKwjAM/aVQsE7ZRSgbCKVzh1mnVK1zjjH0/296c...
6,020a29cf-2c24-478b-8603-c22a90dc3e31,1728,1728,0 1 eNoLCAjJNgIABNkBkg==,[180 118 1908 118 3636 118 5364 118 7092 115 8...,"[(0, 0, 733, 297), (1060, 0, 1303, 85), (1087,...",[eNq1lltvqzAMgP8SDtmZtKcjHWlSN+KulG3QJlzCpdBS+...
554,fea47298-266a-4cf4-93bd-55d1bcc2fc7d,1728,1728,0 1 eNoLCAjJNgIABNkBkg==,[2555936 17 2557664 17 2559392 17 2561090 64 2...,"[(1479, 0, 1728, 341), (736, 31, 1367, 827), (...",[eNqFUMEOgjAM/aU3tzijN02IIxtDEhJPgEajJ/3/m6xlM...
557,ff069fa2-d948-408e-91b3-034cfea428d1,3072,3072,0 1 eNoLCAgIsAQABJ4Beg==,[2377891 60 2380963 60 2384035 60 2387107 60 2...,"[(774, 0, 1674, 402), (1338, 0, 2244, 810), (1...",[eNoLCk2KMMoxDMkzMoCBNL8gowh/ONcrycAFzonzz0Piu...
558,ff23eea9-4bbe-42af-a8da-9ae16321fc6d,2048,2048,0 1 eNoLCAgIMAEABJkBdQ==,[1 268 349 80 2049 268 2397 80 4097 268 4445 8...,"[(0, 0, 244, 528), (180, 0, 504, 636), (728, 0...",[eNq9ksFqwzAMhl8pauPNHTkMVmjxGkUwH3QwtQI+6OD3P...


<font size = '6' >**Helper function**</font>

In [4]:
def binary_mask_to_ascii(mask, mask_val=1):
    """Converts a binary mask into OID challenge encoding ascii text."""
    mask = np.where(mask==mask_val, 1, 0).astype(np.bool)
    
    # check input mask --
    if mask.dtype != np.bool:
        raise ValueError(f"encode_binary_mask expects a binary mask, received dtype == {mask.dtype}")

    mask = np.squeeze(mask)
    if len(mask.shape) != 2:
        raise ValueError(f"encode_binary_mask expects a 2d mask, received shape == {mask.shape}")

    # convert input mask to expected COCO API input --
    mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
    mask_to_encode = mask_to_encode.astype(np.uint8)
    mask_to_encode = np.asfortranarray(mask_to_encode)

    # RLE encode mask --
    encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

    # compress and base64 encoding --
    binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
    base64_str = base64.b64encode(binary_str)
    return base64_str.decode()


def rle_encoding(img, mask_val=1):
    """
    Turns our masks into RLE encoding to easily store them
    and feed them into models later on
    https://en.wikipedia.org/wiki/Run-length_encoding
    
    Args:
        img (np.array): Segmentation array
        mask_val (int): Which value to use to create the RLE
        
    Returns:
        RLE string
    
    """
    dots = np.where(img.T.flatten() == mask_val)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
        
    return ' '.join([str(x) for x in run_lengths])


def rle_to_mask(rle_string, height, width):
    """ Convert RLE sttring into a binary mask 
    
    Args:
        rle_string (rle_string): Run length encoding containing 
            segmentation mask information
        height (int): Height of the original image the map comes from
        width (int): Width of the original image the map comes from
    
    Returns:
        Numpy array of the binary segmentation mask for a given cell
    """
    rows,cols = height,width
    rle_numbers = [int(num_string) for num_string in rle_string.split(' ')]
    rle_pairs = np.array(rle_numbers).reshape(-1,2)
    img = np.zeros(rows*cols,dtype=np.uint8)
    for index,length in rle_pairs:
        index -= 1
        img[index:index+length] = 255
    img = img.reshape(cols,rows)
    img = img.T
    return img


def decode_img(img, img_size=(224,224), testing=False):

    
    # convert the compressed string to a 3D uint8 tensor
    if not testing:
        # resize the image to the desired size
        img = tf.image.decode_png(img, channels=1)
        return tf.cast(tf.image.resize(img, img_size), tf.uint8)
    else:
        return tf.image.decode_png(img, channels=1)
        
      
    
    
def create_pred_col(row):
    """ Simple function to return the correct prediction string
    
    We will want the original public test dataframe submission when it is 
    available. However, we will use the swapped inn submission dataframe
    when it is not.
    
    Args:
        row (pd.Series): A row in the dataframe
    
    Returns:
        The prediction string
    """
    if pd.isnull(row.PredictionString_y):
        return row.PredictionString_x
    else:
        return row.PredictionString_y
    
    
def load_image(img_id, img_dir, testing=False, only_public=False):
    """ Load An Image Using ID and Directory Path - Composes 4 Individual Images """
    if only_public:
        return_axis = -1
        clr_list = ["red", "green", "blue"]
    else:
        return_axis = 0
        clr_list = ["red", "green", "blue", "yellow"]
    
    if not testing:
        rgby = [
            np.asarray(Image.open(os.path.join(img_dir, img_id+f"_{c}.png")), np.uint8) \
            for c in ["red", "green", "blue", "yellow"]
        ]
        return np.stack(rgby, axis=-1)
    else:
        # This is for cellsegmentator
        return np.stack(
            [np.asarray(decode_img(tf.io.read_file(os.path.join(img_dir, img_id+f"_{c}.png")), testing=True), np.uint8)[..., 0] \
             for c in clr_list], axis=return_axis,
        )
        
    
    
def convert_rgby_to_rgb(arr):
    """ Convert a 4 channel (RGBY) image to a 3 channel RGB image.
    
    Advice From Competition Host/User: lnhtrang

    For annotation (by experts) and for the model, I guess we agree that individual 
    channels with full range px values are better. 
    In annotation, we toggled the channels. 
    For visualization purpose only, you can try blending the channels. 
    For example, 
        - red = red + yellow
        - green = green + yellow/2
        - blue=blue.
        
    Args:
        arr (numpy array): The RGBY, 4 channel numpy array for a given image
    
    Returns:
        RGB Image
    """
    
    rgb_arr = np.zeros_like(arr[..., :-1])
    rgb_arr[..., 0] = arr[..., 0]
    rgb_arr[..., 1] = arr[..., 1]+arr[..., 3]/2
    rgb_arr[..., 2] = arr[..., 2]
    
    return rgb_arr

    
    
def flatten_list_of_lists(l_o_l, to_string=False):
    if not to_string:
        return [item for sublist in l_o_l for item in sublist]
    else:
        return [str(item) for sublist in l_o_l for item in sublist]

    

def get_contour_bbox_from_raw(raw_mask):
    """ Get bbox of contour as `xmin ymin xmax ymax`
    
    Args:
        raw_mask (nparray): Numpy array containing segmentation mask information
    
    Returns:
        Numpy array for a cell bounding box coordinates
    """
    cnts = grab_contours(
        cv2.findContours(
            raw_mask, 
            cv2.RETR_EXTERNAL, 
            cv2.CHAIN_APPROX_SIMPLE
        ))
    xywhs = [cv2.boundingRect(cnt) for cnt in cnts]
    xys = [(xywh[0], xywh[1], xywh[0]+xywh[2], xywh[1]+xywh[3]) for xywh in xywhs]
    return sorted(xys, key=lambda x: (x[1], x[0]))


def pad_to_square(a):
    """ Pad an array `a` evenly until it is a square """
    if a.shape[1]>a.shape[0]: # pad height
        n_to_add = a.shape[1]-a.shape[0]
        top_pad = n_to_add//2
        bottom_pad = n_to_add-top_pad
        a = np.pad(a, [(top_pad, bottom_pad), (0, 0), (0, 0)], mode='constant')

    elif a.shape[0]>a.shape[1]: # pad width
        n_to_add = a.shape[0]-a.shape[1]
        left_pad = n_to_add//2
        right_pad = n_to_add-left_pad
        a = np.pad(a, [(0, 0), (left_pad, right_pad), (0, 0)], mode='constant')
    else:
        pass
    return a



def grab_contours(cnts):
    # if the length the contours tuple returned by cv2.findContours
    # is '2' then we are using either OpenCV v2.4, v4-beta, or
    # v4-official
    if len(cnts) == 2:
        cnts = cnts[0]

    # if the length of the contours tuple is '3' then we are using
    # either OpenCV v3, v4-pre, or v4-alpha
    elif len(cnts) == 3:
        cnts = cnts[1]

    # otherwise OpenCV has changed their cv2.findContours return
    # signature yet again and I have no idea WTH is going on
    else:
        raise Exception(("Contours tuple must have length 2 or 3, "
            "otherwise OpenCV changed their cv2.findContours return "
            "signature yet again. Refer to OpenCV's documentation "
            "in that case"))

    # return the actual contours array
    return cnts

    
def tta(original_img_batch, repeats=4):
    """ Perform test time augmentation """
    tta_img_batches = [original_img_batch,]

    for i in range(repeats):
        # create new image batch (tf automatically deep copies)
        img_batch = original_img_batch
        
        SEED = tf.random.uniform((2,), minval=0, maxval=100, dtype=tf.dtypes.int32)
        K = tf.random.uniform((1,), minval=0, maxval=4, dtype=tf.dtypes.int32)[0]

        img_batch = tf.image.stateless_random_flip_left_right(img_batch, SEED)
        img_batch = tf.image.stateless_random_flip_up_down(img_batch, SEED)
        img_batch = tf.image.rot90(img_batch, K)

        img_batch = tf.image.stateless_random_saturation(img_batch, 0.9, 1.1, SEED)
        img_batch = tf.image.stateless_random_brightness(img_batch, 0.075, SEED)
        img_batch = tf.image.stateless_random_contrast(img_batch, 0.9, 1.1, SEED)    
        tta_img_batches.append(img_batch)
    
    return tta_img_batches

<font size = '6' >**Load the model**</font>

In [5]:
# Load inference model
inference_model = tf.keras.models.load_model(main_model,compile=False)

# Parameters
IMAGE_SIZES = [1728, 2048, 3072, 4096]
BATCH_SIZE = 8
CONF_THRESH = 0.0
TILE_SIZE = (224,224)


# Switch what we will be actually infering on
if ONLY_PUBLIC:
    # Make subset dataframes
    predict_df_1728 = pub_ss_df[pub_ss_df.ImageWidth==IMAGE_SIZES[0]]
    predict_df_2048 = pub_ss_df[pub_ss_df.ImageWidth==IMAGE_SIZES[1]]
    predict_df_3072 = pub_ss_df[pub_ss_df.ImageWidth==IMAGE_SIZES[2]]
    predict_df_4096 = pub_ss_df[pub_ss_df.ImageWidth==IMAGE_SIZES[3]]
else:
    # Load Segmentator
    segmentator = cellsegmentator.CellSegmentator(NUC_MODEL, CELL_MODEL, scale_factor=0.25, padding=True)
    
    # Make subset dataframes
    predict_df_1728 = ss_df[ss_df.ImageWidth==IMAGE_SIZES[0]]
    predict_df_2048 = ss_df[ss_df.ImageWidth==IMAGE_SIZES[1]]
    predict_df_3072 = ss_df[ss_df.ImageWidth==IMAGE_SIZES[2]]
    predict_df_4096 = ss_df[ss_df.ImageWidth==IMAGE_SIZES[3]]


predict_ids_1728 = predict_df_1728.ID.to_list()
predict_ids_2048 = predict_df_2048.ID.to_list()
predict_ids_3072 = predict_df_3072.ID.to_list()
predict_ids_4096 = predict_df_4096.ID.to_list()

<font size = '6' >**Ensemble inference**</font>

In [6]:
predictions = []
sub_df = pd.DataFrame(columns=["ID"], data=predict_ids_1728+predict_ids_2048+predict_ids_3072+predict_ids_4096)


for size_idx, submission_ids in enumerate([predict_ids_1728, predict_ids_2048, predict_ids_3072, predict_ids_4096]):
    size = IMAGE_SIZES[size_idx]
    if submission_ids==[]:
        print(f"\n...SKIPPING SIZE {size} AS THERE ARE NO IMAGE IDS ...\n")
        continue
    else:
        print(f"\n...WORKING ON IMAGE IDS FOR SIZE {size} ...\n")
    for i in tqdm(range(0, len(submission_ids), BATCH_SIZE), total=int(np.ceil(len(submission_ids)/BATCH_SIZE))):
        
        # Step 0: Get batch of images as numpy arrays
        batch_rgby_images = [
            load_image(ID, TEST_IMG_DIR, testing=True, only_public=ONLY_PUBLIC) \
            for ID in submission_ids[i:(i+BATCH_SIZE)]
        ]
        
        if ONLY_PUBLIC:
            # Step 1: Get Bounding Boxes
            batch_cell_bboxes = pub_ss_df[pub_ss_df.ID.isin(submission_ids[i:(i+BATCH_SIZE)])].mask_bboxes.values
            
            # Step 2: Get RGB Images (which are actually just labelled as RGBY)
            batch_rgb_images = batch_rgby_images
            
            # Step 3: Get Submission RLEs
            submission_rles = pub_ss_df[pub_ss_df.ID.isin(submission_ids[i:(i+BATCH_SIZE)])].mask_sub_rles.values
            
            # Optional Step: Get the Masks
            if IS_DEMO:
                batch_masks = [
                    sum([rle_to_mask(mask, size, size) for mask in batch]) \
                    for batch in pub_ss_df[pub_ss_df.ID.isin(submission_ids[i:(i+BATCH_SIZE)])].mask_rles.values
                ]
                
            
        else:
            # Step 1: Do Prediction On Batch
            cell_segmentations = segmentator.pred_cells([[rgby_image[j] for rgby_image in batch_rgby_images] for j in [0, 3, 2]])
            nuc_segmentations = segmentator.pred_nuclei([rgby_image[2] for rgby_image in batch_rgby_images])

            # Step 2: Perform Cell Labelling on Batch
            batch_masks = [label_cell(nuc_seg, cell_seg)[1].astype(np.uint8) for nuc_seg, cell_seg in zip(nuc_segmentations, cell_segmentations)]

            # Step 3: Reshape the RGBY Images so They Are Channels Last Across the Batch
            batch_rgb_images = [rgby_image.transpose(1,2,0)[..., :-1] for rgby_image in batch_rgby_images]

            # Step 4: Get Bounding Boxes For All Cells in All Images in Batch
            batch_cell_bboxes = [get_contour_bbox_from_raw(mask) for mask in batch_masks]
            
            # Step 5: Generate Submission RLEs For the Batch
            submission_rles = [[binary_mask_to_ascii(mask, mask_val=cell_id) for cell_id in range(1, mask.max()+1)] for mask in batch_masks]
    
        # Step 6: Cut Out, Pad to Square, and Resize to 224x224
        batch_cell_tiles = [[
            cv2.resize(
                pad_to_square(
                    rgb_image[bbox[1]:bbox[3], bbox[0]:bbox[2], ...]), 
                TILE_SIZE, interpolation=cv2.INTER_CUBIC) for bbox in bboxes] 
            for bboxes, rgb_image in zip(batch_cell_bboxes, batch_rgb_images)
        ]

        # Step 7: (OPTIONAL) Test Time Augmentation
        if DO_TTA:
            tta_batch_cell_tiles = [tta(tf.cast(ct, dtype=tf.float32), repeats=TTA_REPEATS) for ct in batch_cell_tiles]
        else:
            batch_cell_tiles = [tf.cast(ct, dtype=tf.float32) for ct in batch_cell_tiles]
        
        # Step 8: Perform Inference 
        if DO_TTA:
            tta_batch_o_preds = [[inference_model.predict(ct) for ct in bct] for bct in tta_batch_cell_tiles]
            batch_o_preds = [tf.keras.layers.Average()(tta_o_preds).numpy() for tta_o_preds in tta_batch_o_preds]
        else:
            batch_o_preds = [inference_model.predict(cell_tiles) for cell_tiles in batch_cell_tiles]
            
        # Step 9: Post-Process
        batch_confs = [[pred[np.where(pred>CONF_THRESH)] for pred in o_preds] for o_preds in batch_o_preds]
        batch_preds = [[np.where(pred>CONF_THRESH)[0] for pred in o_preds] for o_preds in batch_o_preds]

        for j, preds in enumerate(batch_preds):
            for k in range(len(preds)):
                if preds[k].size==0:
                    batch_preds[j][k]=np.array([18,])
                    batch_confs[j][k]=np.array([1-np.max(batch_o_preds[j][k]),])

        
        # Step 10: Format Predictions To Create Prediction String Easily
        submission_rles = [flatten_list_of_lists([[m,]*len(p) for m, p in zip(masks, preds)]) for masks, preds in zip(submission_rles, batch_preds)]
        batch_preds = [flatten_list_of_lists(preds, to_string=True) for preds in batch_preds]
        batch_confs = [[f"{conf:.4f}" for cell_confs in confs for conf in cell_confs] for confs in batch_confs]
        
        # Step 11: Save Predictions to Be Added to Dataframe At The End
        predictions.extend([" ".join(flatten_list_of_lists(zip(*[preds,confs,masks]))) for preds, confs, masks in zip(batch_preds, batch_confs, submission_rles)])
sub_df["PredictionString"] = predictions

print("\n... TEST DATAFRAME ...\n")
display(sub_df.head(3))


...WORKING ON IMAGE IDS FOR SIZE 1728 ...



  0%|          | 0/1 [00:00<?, ?it/s]


...WORKING ON IMAGE IDS FOR SIZE 2048 ...



  0%|          | 0/1 [00:00<?, ?it/s]


...WORKING ON IMAGE IDS FOR SIZE 3072 ...



  0%|          | 0/1 [00:00<?, ?it/s]


...SKIPPING SIZE 4096 AS THERE ARE NO IMAGE IDS ...


... TEST DATAFRAME ...



,ID,PredictionString
0,020a29cf-2c24-478b-8603-c22a90dc3e31,0 0.0104 eNq1lltvqzAMgP8SDtmZtKcjHWlSN+KulG3QJ...
1,fea47298-266a-4cf4-93bd-55d1bcc2fc7d,0 0.0048 eNqFUMEOgjAM/aU3tzijN02IIxtDEhJPgEajJ...
2,0040581b-f1f2-4fbe-b043-b6bfea5404bb,0 0.0339 eNqtVL0OgyAQfiVIz4TBoYODiVdKE2odGDqYR...


In [7]:
ss_df = ss_df.merge(sub_df, how="left", on="ID")
ss_df["PredictionString"] = ss_df.apply(create_pred_col, axis=1)
ss_df = ss_df.drop(columns=["PredictionString_x", "PredictionString_y"])
display(ss_df)

torch.cuda.empty_cache()

,ID,ImageWidth,ImageHeight,PredictionString
0,0040581b-f1f2-4fbe-b043-b6bfea5404bb,2048,2048,0 0.0339 eNqtVL0OgyAQfiVIz4TBoYODiVdKE2odGDqYR...
1,0173029a-161d-40ef-af28-2342915b22fb,3072,3072,0 0.0002 eNq9VMEKwjAM/aVQsE7ZRSgbCKVzh1mnVK1zj...
2,020a29cf-2c24-478b-8603-c22a90dc3e31,1728,1728,0 0.0104 eNq1lltvqzAMgP8SDtmZtKcjHWlSN+KulG3QJ...
3,fea47298-266a-4cf4-93bd-55d1bcc2fc7d,1728,1728,0 0.0048 eNqFUMEOgjAM/aU3tzijN02IIxtDEhJPgEajJ...
4,ff069fa2-d948-408e-91b3-034cfea428d1,3072,3072,0 0.0307 eNoLCk2KMMoxDMkzMoCBNL8gowh/ONcrycAFz...
5,ff23eea9-4bbe-42af-a8da-9ae16321fc6d,2048,2048,0 0.0010 eNq9ksFqwzAMhl8pauPNHTkMVmjxGkUwH3Qwt...


In [8]:
sub_df = pd.read_csv('../input/hpa-single-cell-image-classification/sample_submission.csv')

def build_decoder(with_labels=True, target_size=(300, 300), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img

    def decode_with_labels(path, label):
        return decode(path), label

    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        return img

    def augment_with_labels(img, label):
        return augment(img), label

    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)

    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)

    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)

    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)

    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)

    return dset

COMPETITION_NAME = "hpa-single-cell-image-classification"
strategy = tf.distribute.get_strategy()
BATCH_SIZE = strategy.num_replicas_in_sync * 16

IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600)

load_dir = f"/kaggle/input/{COMPETITION_NAME}/"
sub_df = pd.read_csv('../input/hpa-single-cell-image-classification/sample_submission.csv')
sub_df = ss_df.copy()

sub_df = sub_df.drop(sub_df.columns[1:],axis=1)

for i in range(19):
    sub_df[f'{i}'] = pd.Series(np.zeros(sub_df.shape[0]))


test_paths = load_dir + "/test/" + sub_df['ID'] + '_green.png'
# Get the multi-labels
label_cols = sub_df.columns[1:]

test_decoder = build_decoder(with_labels=False, target_size=(IMSIZE[7], IMSIZE[7]))
dtest = build_dataset(
    test_paths, bsize=BATCH_SIZE, repeat=False, 
    shuffle=False, augment=False, cache=False,
    decode_fn=test_decoder
)

with strategy.scope():
    model = tf.keras.models.load_model(auxiliary_model)

sub_df[label_cols] = model.predict(dtest, verbose=1)

sub_df.head()

ss_df = pd.merge(ss_df, sub_df, on = 'ID', how = 'left')

for i in range(ss_df.shape[0]):
    if ss_df.loc[i,'PredictionString'] == '0 1 eNoLCAgIMAEABJkBdQ==':
        continue
    a = ss_df.loc[i,'PredictionString']
    b = a.split()
    for j in range(int(len(a.split())/3)):
        for k in range(19):
            if int(b[0 + 3 * j]) == k:

                c = b[0 + 3 * j + 1]               
                b[0 + 3 * j + 1] = str(ss_df.loc[i,f'{k}'] * 0.6 + float(c) * 0.4)

    ss_df.loc[i,'PredictionString'] = ' '.join(b)

ss_df = ss_df[['ID','ImageWidth','ImageHeight','PredictionString']]
ss_df.to_csv('submission.csv',index = False)

1/1 [==============================] - 22s 22s/step


In [9]:
ss_df

,ID,ImageWidth,ImageHeight,PredictionString
0,0040581b-f1f2-4fbe-b043-b6bfea5404bb,2048,2048,0 0.06094576412200928 eNqtVL0OgyAQfiVIz4TBoYOD...
1,0173029a-161d-40ef-af28-2342915b22fb,3072,3072,0 0.06599916680335997 eNq9VMEKwjAM/aVQsE7ZRSgb...
2,020a29cf-2c24-478b-8603-c22a90dc3e31,1728,1728,0 0.005497885856628418 eNq1lltvqzAMgP8SDtmZtKc...
3,fea47298-266a-4cf4-93bd-55d1bcc2fc7d,1728,1728,0 0.011608425064086913 eNqFUMEOgjAM/aU3tzijN02...
4,ff069fa2-d948-408e-91b3-034cfea428d1,3072,3072,0 0.5375168211746215 eNoLCk2KMMoxDMkzMoCBNL8go...
5,ff23eea9-4bbe-42af-a8da-9ae16321fc6d,2048,2048,0 0.0007910481929779053 eNq9ksFqwzAMhl8pauPNHT...
